In [14]:
from functions import *
from output_classes import *

In [15]:
def fill_pts3d(img_db, punts3D_db, pts_cloud, kp_img, matches, images, idx1, idx2):
    num_pt = 0
    for p in pts_cloud.T:
        if img_db[idx1+1].point3D_idxs[matches[(idx1,idx2)][num_pt].queryIdx] != -1:
            num_pt += 1
            continue
        point_id = len(punts3D_db)
        im_pos = kp_img[matches[(idx1, idx2)][num_pt].queryIdx].pt
        im_pts_idx = np.concatenate(([(idx1+1, matches[idx1, idx2][num_pt].queryIdx)], [(j+1, m.trainIdx) for j in range(idx1+1, images.shape[0]) for m in matches[idx1, j] if m.queryIdx == matches[(idx1,idx2)][num_pt].queryIdx]))
        punts3D_db[point_id] = Point3D(
            id=point_id,
            xyz=p,
            rgb=images[0, int(im_pos[1]), int(im_pos[0]), :],
            error=0,        
            image_ids=im_pts_idx[:, 0],
            point2D_idxs=im_pts_idx[:, 1],
        )
        for im_pts in im_pts_idx:
            img_db[im_pts[0]].point3D_idxs[im_pts[1]] = point_id
        num_pt += 1


In [16]:
cameras = {}
punts3D = {}

## COLMAP

images, img_db = load_images_from_folder('dinos')

In [17]:
 # Feature extraction
kp, des = feature_extraction_set(images, img_db)

# Feature matching
matches = feature_matching_set(kp, des)

In [18]:
# Inicializar camara
height, width = images.shape[1:3]
K = np.array([  # for dino
    [2360, 0, width / 2],
    [0, 2360, height / 2],
    [0, 0, 1]])

cameras[1] = Camera(
                id=1,
                model="OPENCV",
                width=width,
                height=height,
                params=[2360, 2360, width / 2, height / 2, 0,0,0,0],
            )

In [19]:
 # Fundamental matrix
pts1 = np.transpose([kp[0][m.queryIdx].pt for m in matches[(0, 1)]])
pts2 = np.transpose([kp[1][m.trainIdx].pt for m in matches[(0, 1)]])

F = eight_point_algorithm(pts1, pts2)

# Essential matrix
E = essential_from_fundamental(K, F, K)  # In this case, the same intrinsic values apply to all images

# Get camera extrinsics from Essential matrix
RT2s = pose_from_essential(E)

In [20]:
# Define RT for camera 1 (center at world origin and matching orientation)
RT1 = np.hstack((np.eye(3), np.zeros((3, 1))))
img_db[1].qvec = rotmat2qvec(np.eye(3))
img_db[1].tvec = np.zeros((3,))

K1, K2 = K, K
RT2 = RT2s[0]

pts3d = np.array([linear_triangulation(K, RT1, K, RT2, pts1, pts2) for RT2 in RT2s])

RT2, pts_cloud = double_disambiguation(K, RT1, K, RT2s, pts1, pts2, pts3d)
pts_cloud = pts_cloud[:, pts_cloud[2, :] > 0]

img_db[2].qvec = rotmat2qvec(RT2[:, :3])
img_db[2].tvec = RT2[:, -1]

In [21]:
punts3D = {}

In [22]:
fill_pts3d(img_db, punts3D, pts_cloud, kp[0], matches, images, 0, 1)

In [23]:
def plot_model(points3d_with_views):
    # Extract the 3D points from points3d_with_views
    pts_cloud = np.array([pt3.xyz for pt3 in points3d_with_views.values() if np.abs(np.sum(pt3.xyz)) < 200])

    # Assuming points_3D is your array of 3D points
    x = pts_cloud[:, 0]
    y = pts_cloud[:, 1]
    z = pts_cloud[:, 2]

    fig = go.Figure(data=[go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=z,                # set color to an array/list of desired values
            colorscale='Viridis',   # choose a colorscale
            opacity=0.8
        )
    )])

    # tight layout
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.show()

In [24]:
plot_model(punts3D)

In [25]:
# for p in pts_cloud.T:
#     point_idx = len(punts3D)
#     index1 = kp[0][matches[(0,1)][point_idx].queryIdx].pt
#     im_pts_idx = np.concatenate(([(1, matches[(0,1)][point_idx].queryIdx)], [(j+1, m.trainIdx) for j in range(1, images.shape[0]) for m in matches[(0,j)] if m.queryIdx == matches[(0,1)][point_idx].queryIdx]))
#     punts3D[point_idx] = Point3D(
#         id=point_idx,
#         xyz=p,
#         rgb=images[0, int(index1[1]), int(index1[0]), :],
#         error=0,        
#         image_ids=im_pts_idx[:, 0],
#         point2D_idxs=im_pts_idx[:, 1],
#     )
#     for im_pts in im_pts_idx:
#         img_db[im_pts[0]].point3D_idxs[im_pts[1]] = point_idx

In [26]:
for i in range(2, len(img_db)):
    # Get projection matrix, rotation and translation
    pts2d = np.array([pt for pt, idx in zip(img_db[i+1].xys, img_db[i+1].point3D_idxs) if idx != -1])
    pts3d = np.array([punts3D[idx].xyz for idx in img_db[i+1].point3D_idxs if idx != -1])
    P = calculate_projection_matrix(K, pts3d, pts2d)
    RT = np.linalg.inv(K) @ P
    img_db[i+1].qvec = rotmat2qvec(RT[:, :3])
    img_db[i+1].tvec = RT[:, -1]
    
    # Triangulate all matches with previous images and
    for j in range(i):
        if len(matches[(j,i)]) == 0:
            continue
        
        pts2d1_idxs = np.array([m.queryIdx for m in matches[(j,i)] if img_db[i+1].point3D_idxs[m.trainIdx] == -1])
        pts2d2_idxs = np.array([m.trainIdx for m in matches[(j,i)] if img_db[i+1].point3D_idxs[m.trainIdx] == -1])

        RTj = np.hstack((qvec2rotmat(img_db[j+1].qvec), img_db[j+1].tvec[:, np.newaxis]))
        RTi = RT

        pts_cloud = linear_triangulation(K, RTj, K, RT, img_db[j+1].xys[pts2d1_idxs, :].T, img_db[i+1].xys[pts2d2_idxs, :].T)

        fill_pts3d(img_db, punts3D, pts_cloud, kp[j], matches, images, j, i)


c:\Users\adria\anaconda3\Lib\site-packages\numpy\linalg\linalg.py:2180: RuntimeWarning:

invalid value encountered in det



LinAlgError: Eigenvalues did not converge

In [ ]:
plot_model(punts3D)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed